In [ ]:
# Follow install instructions from https://github.com/ultralytics/yolov3

In [18]:
import os
import cv2
import glob
import numpy as np
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split
import scipy.io

from numpy import savetxt
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array, save_img
from keras.preprocessing.image import img_to_array

from util.eventvision import read_dataset, read_annotation

# Length of window of time to create a time frame, 10000 = 10ms
TIME_FRAME = 10000

In [19]:
def getEventData(filename):
    td = read_dataset(filename)
    
    width = td.width
    height = td.height
    arr = np.array(td.data)
    
    matrix = np.zeros((height, width))
    
    for item in arr:
        if item[3] > TIME_FRAME:
            break
        else:
            x = item[0]
            y = item[1]
            matrix[y][x] = item[3]
            
    matrix = (matrix / TIME_FRAME)**2       
    matrix = cv2.resize(matrix, (256, 256))
    img = Image.fromarray(matrix * 255).convert("L")
    
    return width, height, img # np.asarray(img)

def getEventAnnotation(filename):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    width = maxX - minX
    height = maxY - minY
    
    x_center = (minX + maxX) / 2
    y_center = (minY + maxY) / 2
    
    return x_center, y_center, width, height

def getImageData(filename):
    img = load_img(filename)
    
    data = img_to_array(img)
    arr = np.array(data)
    width = arr.shape[1]
    height = arr.shape[0]
    
    arr = cv2.resize(arr, (256,256))
    
    return width, height, arr

def getImageAnnotation(filename):
    mat = scipy.io.loadmat(filename)
    data = np.array(mat["box_coord"][0])
    minX = data[2]
    maxX = data[3]
    minY = data[0]
    maxY = data[1]
    
    width = maxX - minX
    height = maxY - minY
    
    x_center = (minX + maxX) / 2
    y_center = (minY + maxY) / 2
    
    return x_center, y_center, width, height

In [21]:
def parse_partial_dataset(getData, getAnnotation, event=True, folders=["airplanes", "car_side", "helicopter", "Motorbikes"]):
    train_points = []
    train_labels = []
    if event:
        event_folders = glob.glob(os.getcwd() + r'\..\Caltech101\*')
        ann_folders = glob.glob(os.getcwd() + r'\..\Caltech101_annotations\*')
    else:
        event_folders = glob.glob(os.getcwd() + r'\..\Caltech101_real\caltech101_objects\101_ObjectCategories\*')
        ann_folders = glob.glob(os.getcwd() + r'\..\Caltech101_real\caltech101_annotations\Annotations\*')
    
    event_folders = [s for s in event_folders if any(xs in s for xs in folders)]
    ann_folders = [s for s in ann_folders if any(xs in s for xs in folders)]
    
    length = len(event_folders)

    for i, _ in enumerate(event_folders):
        events = glob.glob(event_folders[i] +"\\*")
        annotation = glob.glob(ann_folders[i] + "\\*")
        
        for j, _ in enumerate(events):
            w, h, mtrx = getData(events[j])
            x_center, y_center, width, height = getAnnotation(annotation[j])
            
            label = np.zeros(5)
            label[-4:] = np.clip(np.array([x_center / w, y_center / h, width / w, height / h]), 0, 1)
            label[0] = i
            
            train_points.append(mtrx)
            train_labels.append(label)
    
    return train_test_split(train_points, train_labels, test_size=0.2, random_state=42)

In [22]:
def parse_total_dataset(getData, getAnnotation, event=True):
    train_points = []
    train_labels = []
    if event:
        event_folders = glob.glob(os.getcwd() + r'\..\Caltech101\*')
        ann_folders = glob.glob(os.getcwd() + r'\..\Caltech101_annotations\*')
    else:
        event_folders = glob.glob(os.getcwd() + r'\..\Caltech101_real\caltech101_objects\101_ObjectCategories\*')
        ann_folders = glob.glob(os.getcwd() + r'\..\Caltech101_real\caltech101_annotations\Annotations\*')
    
    length = len(event_folders)

    for i, name in enumerate(event_folders):
        folder_names.append("'" + name + "'")
        
        events = glob.glob(event_folders[i] +"\\*")
        annotation = glob.glob(ann_folders[i] + "\\*")
        
        for j, _ in enumerate(events):
            w, h, mtrx = getData(events[j])
            x_center, y_center, width, height = getAnnotation(annotation[j])
            
            label = np.zeros(5)
            label[-4:] = np.clip(np.array([x_center / w, y_center / h, width / w, height / h]), 0, 1)
            label[0] = i
            
            train_points.append(mtrx)
            train_labels.append(label)
            
    return train_test_split(train_points, train_labels, test_size=0.2, random_state=42)

# parse_total_dataset(getEventData, getEventAnnotation)

folder_names = []
event_folders = glob.glob(os.getcwd() + r'\..\Caltech101\*')
for i, name in enumerate(event_folders):
    folder_names.append(name.split(os.sep)[-1])
    
print(folder_names, len(folder_names))

['accordion', 'airplanes', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car_side', 'ceiling_fan', 'cellphone', 'chair', 'chandelier', 'cougar_body', 'cougar_face', 'crab', 'crayfish', 'crocodile', 'crocodile_head', 'cup', 'dalmatian', 'dollar_bill', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'Faces_easy', 'ferry', 'flamingo', 'flamingo_head', 'garfield', 'gerenuk', 'gramophone', 'grand_piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline_skate', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'Leopards', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'Motorbikes', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster', 'saxophone', 'schooner', 'scissors', 'scorpion', 'sea_horse', 'snoopy', 'soccer_ball', 'stapler', 'starf

In [23]:
def create_ncaltech_yolo_dataset():
    X_train, X_val, y_train, y_val = parse_total_dataset(getEventData, getEventAnnotation)
    img_train_dir = os.getcwd() + "\\NCaltech101\\images\\train\\"
    img_test_dir = os.getcwd() + "\\NCaltech101\\images\\val\\"
    
    label_train_dir = os.getcwd() + "\\NCaltech101\\labels\\train\\"
    label_test_dir = os.getcwd() + "\\NCaltech101\\labels\\val\\"
    
    
    for i, file in enumerate(X_train):
        X_train[i].save(img_train_dir + 'x_' + str(i) + '.jpg')
        
        np.savetxt(label_train_dir + 'x_' + str(i) + '.txt', (y_train[i], ), delimiter=' ',fmt='%g')
            
    for i, file in enumerate(X_val):
        X_val[i].save(img_test_dir + 'x_' + str(i) + '.jpg')
        
        np.savetxt(label_test_dir + 'x_' + str(i) + '.txt', (y_val[i], ), delimiter=' ',fmt='%g')
        
    
create_ncaltech_yolo_dataset()

In [ ]:
# train_command = python train.py --img 256 --batch 40 --epochs 20 --data NCaltech101.yaml --weights yolov3.pt

# test_command = python val.py --img 256 --task val --data NCaltech101.yaml --iou 0.7 --weights runs\train\event\weights\best.pt --save-txt --save-json

In [ ]:
# TESTING EVENTS 10 ms time window

# train data
# 20 epochs completed in 0.080 hours
# Model Summary: 261 layers, 61513585 parameters, 0 gradients, 154.8 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all        362        362      0.929      0.909      0.927      0.579
#             airplane        362        167      0.959      0.982       0.98      0.714
#                  car        362         23      0.916      0.957      0.986      0.571
#           helicopter        362         17      0.856      0.706      0.747      0.226
#            motorbike        362        155      0.985      0.994      0.995      0.806
            
# test data
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all        362        362      0.929      0.909      0.922      0.576
#             airplane        362        167      0.959      0.982      0.985      0.715
#                  car        362         23      0.917      0.956      0.982      0.568
#           helicopter        362         17      0.857      0.706      0.726      0.213
#            motorbike        362        155      0.985      0.994      0.995      0.806
# Speed: 0.1ms pre-process, 4.8ms inference, 1.0ms NMS per image at shape (32, 3, 256, 256)

In [ ]:
# TESTING EVENTS 25 ms time window

# train data
# 20 epochs completed in 0.078 hours
# Model Summary: 261 layers, 61513585 parameters, 0 gradients, 154.8 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all        362        362      0.941      0.935      0.958      0.747
#             airplane        362        167      0.943      0.994       0.99      0.828
#                  car        362         23      0.958      0.981      0.993      0.844
#           helicopter        362         17      0.866      0.765      0.854      0.441
#            motorbike        362        155      0.997          1      0.995      0.877

# test data
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all        362        362      0.927      0.935      0.953      0.744
#             airplane        362        167      0.943      0.994       0.99      0.827
#                  car        362         23      0.958      0.981      0.993      0.838
#           helicopter        362         17      0.812      0.764      0.833      0.433
#            motorbike        362        155      0.997          1      0.995      0.876
# Speed: 0.1ms pre-process, 5.3ms inference, 1.0ms NMS per image at shape (32, 3, 256, 256)

In [20]:
# TESTING EVENTS 50 ms time window

# train data
# 20 epochs completed in 0.081 hours
# Model Summary: 261 layers, 61513585 parameters, 0 gradients, 154.8 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all        362        362      0.947      0.969      0.973      0.793
#             airplane        362        167      0.962      0.994       0.99      0.832
#                  car        362         23      0.888          1      0.995      0.875
#           helicopter        362         17      0.937      0.882      0.913       0.58
#            motorbike        362        155          1      0.999      0.995      0.886

# test data
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all        362        362       0.94      0.969      0.972      0.793
#             airplane        362        167      0.962      0.994       0.99      0.833
#                  car        362         23      0.863          1      0.995      0.875
#           helicopter        362         17      0.937      0.881      0.907      0.577
#            motorbike        362        155          1      0.999      0.995      0.886
# Speed: 0.1ms pre-process, 5.9ms inference, 1.1ms NMS per image at shape (32, 3, 256, 256)

In [11]:
def create_caltech_yolo_dataset():
    X_train, X_val, y_train, y_val = parse_total_dataset(getImageData, getImageAnnotation, False)
    img_train_dir = os.getcwd() + "\\Caltech101\\images\\train\\"
    img_test_dir = os.getcwd() + "\\Caltech101\\images\\val\\"
    
    label_train_dir = os.getcwd() + "\\Caltech101\\labels\\train\\"
    label_test_dir = os.getcwd() + "\\Caltech101\\labels\\val\\"
    
    
    for i, file in enumerate(X_train):
        save_img(img_train_dir + 'x_' + str(i) + '.jpg', X_train[i])
        
        np.savetxt(label_train_dir + 'x_' + str(i) + '.txt', (y_train[i], ), delimiter=' ',fmt='%g')
            
    for i, file in enumerate(X_val):
        save_img(img_test_dir + 'x_' + str(i) + '.jpg', X_val[i])
        
        np.savetxt(label_test_dir + 'x_' + str(i) + '.txt', (y_val[i], ), delimiter=' ',fmt='%g')
        
    
create_caltech_yolo_dataset()

<ipython-input-2-952b9ae16337>:62: RuntimeWarning: overflow encountered in ubyte_scalars
  x_center = (minX + maxX) / 2
<ipython-input-2-952b9ae16337>:63: RuntimeWarning: overflow encountered in ubyte_scalars
  y_center = (minY + maxY) / 2


In [ ]:
# TESTING Images

# train_command = python train.py --img 256 --batch 40 --epochs 20 --data Caltech101.yaml --weights yolov3.pt

# test_command = python val.py --img 256 --task val --data Caltech101.yaml --iou 0.7 --weights runs\train\image\weights\best.pt --save-txt --save-json

# train data
# 20 epochs completed in 0.081 hours
# Model Summary: 261 layers, 61513585 parameters, 0 gradients, 154.8 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all        362        362      0.849      0.867      0.925      0.712
#             airplane        362        167          1      0.995      0.995      0.909
#                  car        362         23      0.823      0.609      0.792      0.544
#           helicopter        362         17      0.793      0.903      0.956      0.597
#            motorbike        362        155      0.778      0.961      0.958      0.799

# test data
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all        362        362       0.83      0.867      0.925      0.711
#             airplane        362        167          1      0.995      0.995      0.911
#                  car        362         23      0.823      0.608      0.797      0.544
#           helicopter        362         17      0.719      0.903      0.949      0.593
#            motorbike        362        155      0.778      0.961      0.957      0.797
# Speed: 0.1ms pre-process, 5.5ms inference, 1.1ms NMS per image at shape (32, 3, 256, 256)

In [ ]:
# Testing the entire dataset: 
# python train.py --img 256 --batch 40 --epochs 40 --data NCaltech101.yaml --weights yolov3.pt
# python train.py --img 256 --batch 40 --epochs 40 --data Caltech101.yaml --weights yolov3.pt

In [ ]:
# Testing full NCaltech 10ms time window

# 40 epochs completed in 0.633 hours.
# Model Summary: 261 layers, 62030545 parameters, 0 gradients, 156.4 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all       1649       1649      0.519      0.472      0.457      0.302
#            accordion       1649         11      0.773      0.932      0.956      0.674
#            airplanes       1649        168      0.976      0.974      0.983      0.805
#               anchor       1649          8      0.489      0.122       0.26      0.129
#                  ant       1649          4       0.23       0.25      0.184      0.111
#               barrel       1649          6      0.208      0.333      0.391      0.238
#                 bass       1649         17       0.32      0.176      0.242      0.157
#               beaver       1649         14          1          0     0.0271    0.00688
#            binocular       1649          3      0.227      0.333      0.104     0.0553
#               bonsai       1649         26      0.575      0.538      0.534       0.32
#                brain       1649         20      0.489       0.65      0.526      0.389
#         brontosaurus       1649          7      0.228      0.143       0.16     0.0941
#               buddha       1649         19      0.248      0.526      0.392      0.281
#            butterfly       1649         22      0.566      0.772      0.701      0.496
#               camera       1649          8      0.279       0.75      0.686      0.345
#               cannon       1649         11          1          0     0.0605     0.0199
#             car_side       1649         27      0.612          1      0.995      0.757
#          ceiling_fan       1649          8      0.349        0.5      0.503      0.448
#            cellphone       1649         15      0.541        0.8       0.83      0.465
#                chair       1649         14      0.238      0.357      0.341      0.274
#           chandelier       1649         14      0.287      0.643       0.45      0.294
#          cougar_body       1649          9          1          0      0.166     0.0869
#          cougar_face       1649         13      0.631      0.615      0.563      0.349
#                 crab       1649         18      0.488      0.111      0.192     0.0593
#             crayfish       1649         13      0.315      0.231      0.279      0.106
#            crocodile       1649         11      0.498      0.181      0.228     0.0708
#       crocodile_head       1649          5          1          0     0.0784     0.0237
#                  cup       1649         10      0.128        0.4     0.0689      0.037
#            dalmatian       1649         13      0.331      0.153       0.12     0.0497
#          dollar_bill       1649          8        0.7          1      0.928       0.75
#              dolphin       1649         10      0.291        0.3      0.189      0.072
#            dragonfly       1649         13      0.511      0.692      0.661      0.479
#      electric_guitar       1649          8      0.178      0.625      0.311      0.187
#             elephant       1649         13      0.272      0.154      0.176     0.0735
#                  emu       1649          6          0          0     0.0973     0.0241
#            euphonium       1649         15       0.76        0.8      0.801      0.556
#                 ewer       1649         22      0.556      0.727      0.661      0.458
#           Faces_easy       1649         91      0.967      0.934      0.979      0.753
#                ferry       1649         13      0.635      0.538      0.593      0.263
#             flamingo       1649         11      0.164      0.182      0.113     0.0436
#        flamingo_head       1649          5          0          0     0.0258     0.0133
#             garfield       1649          8      0.432      0.875      0.701      0.496
#              gerenuk       1649          4          1          0     0.0179    0.00606
#           gramophone       1649         18      0.292      0.111      0.152      0.125
#          grand_piano       1649         27      0.974      0.889      0.958      0.802
#            hawksbill       1649         20      0.351       0.46      0.374      0.185
#            headphone       1649          5      0.249        0.8      0.811      0.519
#             hedgehog       1649          9       0.34      0.556      0.405      0.154
#           helicopter       1649         14      0.473      0.714      0.613      0.232
#                 ibis       1649         16      0.246       0.25      0.224       0.11
#         inline_skate       1649          9      0.628      0.556      0.647      0.524
#          joshua_tree       1649         15      0.359        0.4      0.505      0.273
#             kangaroo       1649         19       0.49      0.316      0.319     0.0905
#                ketch       1649         29      0.536      0.828      0.685      0.415
#                 lamp       1649         10      0.453      0.498      0.379      0.242
#               laptop       1649         20      0.756        0.6      0.829      0.484
#             Leopards       1649         36      0.561      0.745      0.732      0.275
#                llama       1649         24      0.742     0.0833      0.237     0.0725
#              lobster       1649          8          1          0     0.0758     0.0407
#                lotus       1649         11     0.0808     0.0909     0.0462     0.0192
#             mandolin       1649          6      0.286      0.869      0.555      0.481
#               mayfly       1649          6      0.645      0.333      0.279     0.0916
#              menorah       1649         18      0.941      0.611      0.626      0.512
#            metronome       1649          7          0          0      0.107     0.0387
#              minaret       1649         12      0.647      0.833      0.828      0.428
#           Motorbikes       1649        161      0.988      0.996      0.995      0.817
#             nautilus       1649          7      0.348      0.429      0.379      0.342
#              octopus       1649          6          1          0     0.0423     0.0172
#                okapi       1649         10      0.364        0.5      0.445      0.241
#               pagoda       1649         10      0.617      0.647      0.757       0.44
#                panda       1649          9          1          0     0.0415     0.0254
#               pigeon       1649          5          0          0    0.00594    0.00239
#                pizza       1649          9      0.385      0.556       0.47       0.33
#             platypus       1649          4          1          0      0.248      0.247
#              pyramid       1649         16      0.479       0.25      0.333      0.141
#             revolver       1649         15      0.623      0.867      0.864       0.62
#                rhino       1649          8      0.192        0.5      0.228      0.147
#              rooster       1649         17       0.74     0.0588      0.274      0.219
#            saxophone       1649          9      0.545      0.667      0.667      0.485
#             schooner       1649         13       0.35      0.769      0.532      0.361
#             scissors       1649          6      0.648      0.833      0.931      0.575
#             scorpion       1649         20          1          0      0.215       0.12
#            sea_horse       1649         11      0.773      0.315       0.35      0.181
#               snoopy       1649         10      0.683        0.9      0.886      0.617
#          soccer_ball       1649         18      0.523      0.791      0.825      0.684
#              stapler       1649          5      0.157        0.6      0.607      0.562
#             starfish       1649         13      0.466      0.462       0.43      0.302
#          stegosaurus       1649         11       0.59      0.636       0.64      0.413
#            stop_sign       1649         15      0.541        0.8      0.795      0.728
#           strawberry       1649          4       0.49        0.5      0.355      0.186
#            sunflower       1649         11      0.229      0.273      0.213      0.154
#                 tick       1649         10      0.475        0.5      0.401      0.205
#            trilobite       1649         13      0.725      0.923      0.887      0.425
#             umbrella       1649         17      0.425      0.588      0.603      0.412
#                watch       1649         49      0.916      0.857      0.885      0.689
#          water_lilly       1649          7      0.498      0.429      0.393      0.318
#           wheelchair       1649         13       0.27      0.385      0.247      0.127
#             wild_cat       1649          7          1          0     0.0238    0.00759
#        windsor_chair       1649          8        0.4      0.625      0.596       0.48
#               wrench       1649          7      0.271      0.714      0.539      0.395
#             yin_yang       1649         15      0.641      0.933      0.898      0.762

In [ ]:
# Testing full NCaltech 25ms time window

# 40 epochs completed in 0.639 hours.
# Model Summary: 261 layers, 62030545 parameters, 0 gradients, 156.4 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all       1649       1649      0.762      0.672      0.735      0.552
#            accordion       1649         11          1      0.708      0.995      0.683
#            airplanes       1649        168      0.966      0.988      0.986      0.881
#               anchor       1649          8      0.625        0.5      0.615      0.295
#                  ant       1649          4      0.597      0.742      0.663      0.377
#               barrel       1649          6      0.464        0.5        0.6      0.433
#                 bass       1649         17        0.9      0.529      0.627      0.399
#               beaver       1649         14          1          0      0.174     0.0918
#            binocular       1649          3      0.367      0.333      0.433      0.286
#               bonsai       1649         26      0.916      0.846      0.899      0.678
#                brain       1649         20      0.904          1       0.98      0.869
#         brontosaurus       1649          7      0.279      0.143      0.241      0.164
#               buddha       1649         19          1      0.725       0.91      0.529
#            butterfly       1649         22      0.907          1      0.993      0.788
#               camera       1649          8      0.791      0.875      0.971      0.818
#               cannon       1649         11      0.799      0.361      0.507      0.257
#             car_side       1649         27      0.944          1      0.995      0.911
#          ceiling_fan       1649          8      0.917      0.625       0.72      0.606
#            cellphone       1649         15      0.952      0.933      0.936      0.804
#                chair       1649         14      0.633      0.493      0.614      0.539
#           chandelier       1649         14      0.786      0.786       0.85      0.603
#          cougar_body       1649          9          1          0      0.122      0.088
#          cougar_face       1649         13      0.983      0.923      0.926      0.641
#                 crab       1649         18      0.781      0.111      0.355      0.223
#             crayfish       1649         13          0          0      0.126     0.0586
#            crocodile       1649         11      0.247     0.0909      0.181      0.069
#       crocodile_head       1649          5      0.177        0.2       0.29      0.101
#                  cup       1649         10      0.544        0.7      0.561      0.365
#            dalmatian       1649         13          1      0.897      0.986       0.66
#          dollar_bill       1649          8      0.782          1      0.995      0.736
#              dolphin       1649         10      0.632        0.9       0.82      0.615
#            dragonfly       1649         13      0.914      0.819      0.898      0.695
#      electric_guitar       1649          8      0.481       0.75      0.735      0.393
#             elephant       1649         13      0.343      0.308      0.405      0.268
#                  emu       1649          6      0.287      0.333      0.476      0.315
#            euphonium       1649         15      0.961          1      0.995      0.837
#                 ewer       1649         22      0.895      0.772      0.869      0.729
#           Faces_easy       1649         91      0.995          1      0.995      0.863
#                ferry       1649         13      0.856      0.846      0.879      0.419
#             flamingo       1649         11      0.632      0.545       0.52      0.369
#        flamingo_head       1649          5      0.738       0.57       0.55      0.379
#             garfield       1649          8      0.728      0.625      0.685      0.616
#              gerenuk       1649          4          1          0     0.0527      0.026
#           gramophone       1649         18      0.767      0.548      0.695      0.536
#          grand_piano       1649         27      0.921      0.963      0.964      0.865
#            hawksbill       1649         20      0.605        0.7      0.762      0.413
#            headphone       1649          5      0.498        0.8       0.78      0.557
#             hedgehog       1649          9      0.617      0.778      0.861      0.573
#           helicopter       1649         14      0.812      0.714      0.832      0.366
#                 ibis       1649         16      0.764      0.625      0.785       0.45
#         inline_skate       1649          9      0.879      0.778      0.874      0.713
#          joshua_tree       1649         15       0.83      0.979      0.939      0.678
#             kangaroo       1649         19      0.594      0.309      0.446      0.232
#                ketch       1649         29      0.891      0.897      0.931      0.694
#                 lamp       1649         10      0.758      0.629      0.733      0.599
#               laptop       1649         20      0.951      0.976      0.971       0.77
#             Leopards       1649         36      0.891      0.972      0.938       0.62
#                llama       1649         24      0.552      0.155      0.435       0.29
#              lobster       1649          8          1          0      0.151      0.095
#                lotus       1649         11      0.518      0.818      0.732      0.363
#             mandolin       1649          6      0.781      0.833      0.852      0.731
#               mayfly       1649          6      0.419      0.333      0.421       0.32
#              menorah       1649         18       0.95      0.889      0.896      0.715
#            metronome       1649          7      0.532      0.571      0.675      0.516
#              minaret       1649         12          1       0.95      0.995       0.72
#           Motorbikes       1649        161      0.995          1      0.995      0.889
#             nautilus       1649          7      0.355      0.714      0.684      0.642
#              octopus       1649          6      0.414      0.333      0.389      0.325
#                okapi       1649         10       0.51        0.8      0.801      0.547
#               pagoda       1649         10      0.964        0.9      0.905      0.572
#                panda       1649          9      0.588      0.333      0.452      0.284
#               pigeon       1649          5      0.366      0.579      0.425      0.341
#                pizza       1649          9      0.883      0.889      0.887      0.767
#             platypus       1649          4          1          0      0.495      0.401
#              pyramid       1649         16      0.809      0.938      0.965       0.68
#             revolver       1649         15      0.901          1      0.991      0.807
#                rhino       1649          8      0.575       0.75      0.848      0.638
#              rooster       1649         17          1        0.9      0.957       0.84
#            saxophone       1649          9      0.962      0.889      0.898      0.643
#             schooner       1649         13      0.798      0.846      0.869      0.703
#             scissors       1649          6       0.88          1      0.995      0.888
#             scorpion       1649         20      0.683       0.55       0.57      0.442
#            sea_horse       1649         11          1      0.523      0.738      0.317
#               snoopy       1649         10      0.887        0.9      0.896      0.671
#          soccer_ball       1649         18          1      0.882      0.992      0.915
#              stapler       1649          5      0.755      0.618      0.826      0.496
#             starfish       1649         13      0.688      0.615      0.683      0.583
#          stegosaurus       1649         11      0.956          1      0.995      0.915
#            stop_sign       1649         15      0.958      0.933      0.959      0.887
#           strawberry       1649          4      0.925       0.75      0.749      0.639
#            sunflower       1649         11      0.876          1      0.995      0.809
#                 tick       1649         10      0.821      0.916      0.978      0.844
#            trilobite       1649         13      0.797      0.906      0.811       0.51
#             umbrella       1649         17      0.955      0.824      0.892       0.76
#                watch       1649         49      0.968          1      0.995      0.902
#          water_lilly       1649          7      0.763      0.286       0.65      0.507
#           wheelchair       1649         13      0.788      0.462      0.631      0.344
#             wild_cat       1649          7          1          0      0.193      0.143
#        windsor_chair       1649          8      0.695      0.856      0.827      0.731
#               wrench       1649          7      0.522      0.714      0.832      0.665
#             yin_yang       1649         15      0.893      0.933      0.936      0.814

In [ ]:
# Testing full NCaltech 50ms time window

# 40 epochs completed in 0.635 hours.
# Model Summary: 261 layers, 62030545 parameters, 0 gradients, 156.4 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all       1649       1649      0.777      0.736      0.779      0.607
#            accordion       1649         11      0.914      0.964      0.988      0.799
#            airplanes       1649        168      0.976      0.982      0.987      0.878
#               anchor       1649          8       0.33      0.625      0.573       0.27
#                  ant       1649          4      0.316      0.585      0.454      0.146
#               barrel       1649          6       0.65      0.667      0.635      0.456
#                 bass       1649         17      0.783      0.471      0.627      0.431
#               beaver       1649         14          1          0      0.257      0.161
#            binocular       1649          3          1       0.61      0.806      0.688
#               bonsai       1649         26      0.884      0.923      0.934      0.702
#                brain       1649         20      0.813          1       0.98      0.913
#         brontosaurus       1649          7      0.259      0.571      0.484      0.391
#               buddha       1649         19          1      0.938      0.993      0.705
#            butterfly       1649         22      0.907      0.886      0.963      0.813
#               camera       1649          8      0.609      0.875      0.944      0.779
#               cannon       1649         11      0.738      0.364      0.415      0.247
#             car_side       1649         27      0.965          1      0.995      0.906
#          ceiling_fan       1649          8      0.643       0.75       0.83      0.693
#            cellphone       1649         15      0.911      0.933      0.959      0.749
#                chair       1649         14      0.788      0.796      0.781      0.707
#           chandelier       1649         14      0.691      0.857       0.86      0.694
#          cougar_body       1649          9      0.923      0.111      0.408      0.299
#          cougar_face       1649         13      0.858      0.923      0.914      0.695
#                 crab       1649         18      0.799      0.443       0.62      0.362
#             crayfish       1649         13      0.352      0.169      0.243      0.171
#            crocodile       1649         11      0.261      0.273      0.156     0.0511
#       crocodile_head       1649          5          1      0.539      0.729       0.45
#                  cup       1649         10      0.744        0.9      0.939      0.607
#            dalmatian       1649         13      0.924          1      0.995      0.844
#          dollar_bill       1649          8      0.694          1      0.995      0.806
#              dolphin       1649         10      0.526        0.8      0.713      0.561
#            dragonfly       1649         13      0.964      0.769      0.914      0.733
#      electric_guitar       1649          8      0.581      0.866      0.771      0.414
#             elephant       1649         13      0.522      0.589      0.589      0.398
#                  emu       1649          6      0.467        0.5       0.54       0.45
#            euphonium       1649         15      0.953          1      0.995      0.845
#                 ewer       1649         22      0.971          1      0.995      0.902
#           Faces_easy       1649         91      0.996      0.989      0.995      0.897
#                ferry       1649         13      0.762      0.769      0.801       0.41
#             flamingo       1649         11      0.622      0.748      0.719      0.463
#        flamingo_head       1649          5      0.479        0.4      0.382      0.325
#             garfield       1649          8      0.583      0.875      0.847      0.731
#              gerenuk       1649          4          1          0     0.0647     0.0432
#           gramophone       1649         18      0.942      0.556      0.856      0.648
#          grand_piano       1649         27      0.964      0.999      0.989      0.857
#            hawksbill       1649         20      0.792        0.9      0.912       0.57
#            headphone       1649          5      0.736        0.8      0.866      0.805
#             hedgehog       1649          9      0.779      0.778      0.833      0.563
#           helicopter       1649         14      0.994      0.786      0.845      0.446
#                 ibis       1649         16      0.861      0.625      0.781      0.499
#         inline_skate       1649          9      0.906      0.889      0.886      0.809
#          joshua_tree       1649         15      0.926      0.838      0.959      0.764
#             kangaroo       1649         19      0.735      0.368      0.565      0.299
#                ketch       1649         29      0.923      0.827      0.962      0.791
#                 lamp       1649         10      0.886        0.9      0.945      0.793
#               laptop       1649         20      0.974          1      0.995       0.87
#             Leopards       1649         36       0.88      0.972      0.987      0.635
#                llama       1649         24      0.893      0.347      0.591      0.394
#              lobster       1649          8          1          0      0.403      0.218
#                lotus       1649         11      0.594       0.93      0.798      0.453
#             mandolin       1649          6      0.546          1      0.785      0.691
#               mayfly       1649          6      0.176      0.167      0.259      0.197
#              menorah       1649         18      0.965      0.944      0.992      0.876
#            metronome       1649          7      0.696      0.659      0.742      0.557
#              minaret       1649         12      0.994          1      0.995      0.672
#           Motorbikes       1649        161      0.996          1      0.995      0.914
#             nautilus       1649          7       0.83      0.714      0.857      0.787
#              octopus       1649          6      0.354      0.333      0.315        0.2
#                okapi       1649         10      0.713        0.8      0.843      0.656
#               pagoda       1649         10      0.868        0.9      0.952      0.664
#                panda       1649          9          1      0.723      0.841      0.371
#               pigeon       1649          5       0.31        0.4      0.326      0.264
#                pizza       1649          9      0.814      0.778      0.927      0.803
#             platypus       1649          4      0.489      0.484      0.412      0.278
#              pyramid       1649         16      0.817      0.938      0.918      0.616
#             revolver       1649         15       0.93          1      0.995      0.876
#                rhino       1649          8      0.573      0.675      0.785      0.671
#              rooster       1649         17      0.898      0.941       0.99      0.864
#            saxophone       1649          9      0.938      0.889      0.934      0.694
#             schooner       1649         13      0.573      0.769      0.728       0.59
#             scissors       1649          6      0.724          1      0.995      0.784
#             scorpion       1649         20        0.7      0.585      0.644      0.461
#            sea_horse       1649         11      0.604      0.278      0.439      0.229
#               snoopy       1649         10      0.978        0.9      0.916      0.661
#          soccer_ball       1649         18          1      0.933      0.955      0.914
#              stapler       1649          5      0.529        0.8      0.607      0.446
#             starfish       1649         13      0.821      0.923      0.955      0.786
#          stegosaurus       1649         11      0.912      0.942      0.988      0.967
#            stop_sign       1649         15      0.956          1      0.995      0.946
#           strawberry       1649          4      0.787       0.75      0.945      0.786
#            sunflower       1649         11      0.733          1      0.995      0.792
#                 tick       1649         10      0.921          1      0.995       0.82
#            trilobite       1649         13      0.971          1      0.995       0.67
#             umbrella       1649         17      0.945      0.824      0.848      0.755
#                watch       1649         49      0.965       0.98      0.987      0.895
#          water_lilly       1649          7      0.566      0.286      0.624      0.485
#           wheelchair       1649         13      0.669      0.615      0.609      0.365
#             wild_cat       1649          7          1          0     0.0955     0.0696
#        windsor_chair       1649          8      0.767       0.75      0.839      0.804
#               wrench       1649          7      0.786          1       0.96        0.9
#             yin_yang       1649         15      0.897      0.933      0.939      0.909

In [ ]:
# Testing full Caltech101 images

# 40 epochs completed in 0.639 hours.
# Model Summary: 261 layers, 62030545 parameters, 0 gradients, 156.4 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@
#                  all       1649       1649      0.686      0.765       0.78      0.649
#            accordion       1649         11      0.944          1      0.995      0.928
#            airplanes       1649        168      0.994      0.993      0.995      0.948
#               anchor       1649          8      0.462      0.875      0.789      0.577
#                  ant       1649          4      0.478          1       0.87      0.698
#               barrel       1649          6       0.56        0.5      0.535      0.386
#                 bass       1649         17      0.816      0.765      0.841      0.607
#               beaver       1649         14      0.432      0.218      0.374      0.275
#            binocular       1649          3      0.777          1      0.995      0.808
#               bonsai       1649         26      0.863      0.885      0.946      0.804
#                brain       1649         20      0.794      0.962      0.985       0.93
#         brontosaurus       1649          7      0.697          1      0.918       0.64
#               buddha       1649         19      0.831      0.947      0.925      0.713
#            butterfly       1649         22      0.877      0.818      0.821      0.694
#               camera       1649          8      0.844          1      0.982      0.857
#               cannon       1649         11      0.736      0.455      0.598      0.425
#             car_side       1649         27      0.484          1      0.783      0.655
#          ceiling_fan       1649          8      0.787       0.75      0.776      0.675
#            cellphone       1649         15      0.545      0.559      0.589      0.447
#                chair       1649         14      0.754      0.929      0.921      0.843
#           chandelier       1649         14      0.718      0.857      0.748      0.683
#          cougar_body       1649          9      0.567      0.729      0.802      0.697
#          cougar_face       1649         13      0.623      0.692      0.725      0.646
#                 crab       1649         18      0.671      0.444      0.503      0.319
#             crayfish       1649         13      0.472      0.846      0.678      0.516
#            crocodile       1649         11      0.562      0.455      0.697      0.302
#       crocodile_head       1649          5       0.54        0.8      0.751      0.545
#                  cup       1649         10      0.628        0.8      0.693       0.55
#            dalmatian       1649         13      0.914      0.822      0.851      0.647
#          dollar_bill       1649          8      0.893          1      0.995      0.878
#              dolphin       1649         10      0.638        0.7      0.813      0.684
#            dragonfly       1649         13      0.815      0.681      0.737      0.577
#      electric_guitar       1649          8      0.553          1      0.949      0.747
#             elephant       1649         13      0.378      0.538      0.278      0.233
#                  emu       1649          6       0.28        0.5      0.227      0.171
#            euphonium       1649         15      0.912      0.933       0.96      0.783
#                 ewer       1649         22      0.918      0.955       0.97      0.865
#           Faces_easy       1649         91      0.775      0.813      0.822       0.76
#                ferry       1649         13      0.897      0.923       0.98      0.704
#             flamingo       1649         11      0.872          1      0.995      0.794
#        flamingo_head       1649          5      0.794      0.777      0.752      0.423
#             garfield       1649          8      0.717      0.875      0.865      0.783
#              gerenuk       1649          4      0.468       0.75      0.662      0.483
#           gramophone       1649         18      0.822      0.833       0.91      0.815
#          grand_piano       1649         27      0.902      0.963      0.961      0.871
#            hawksbill       1649         20      0.762          1      0.981      0.755
#            headphone       1649          5      0.462        0.6      0.643      0.566
#             hedgehog       1649          9      0.651      0.778       0.86      0.715
#           helicopter       1649         14      0.676      0.857        0.9      0.706
#                 ibis       1649         16      0.456      0.438      0.489      0.362
#         inline_skate       1649          9        0.7      0.778      0.737      0.659
#          joshua_tree       1649         15      0.573      0.537      0.608      0.476
#             kangaroo       1649         19       0.83      0.772      0.886      0.649
#                ketch       1649         29      0.561      0.724      0.709      0.634
#                 lamp       1649         10      0.837          1       0.94      0.818
#               laptop       1649         20      0.767       0.85      0.874      0.756
#             Leopards       1649         36      0.962          1      0.994       0.87
#                llama       1649         24      0.587      0.593      0.623      0.524
#              lobster       1649          8      0.333      0.194      0.361      0.235
#                lotus       1649         11      0.304      0.359      0.395      0.315
#             mandolin       1649          6      0.495      0.833      0.839      0.789
#               mayfly       1649          6       0.59        0.5       0.62      0.415
#              menorah       1649         18      0.806      0.923      0.918      0.861
#            metronome       1649          7      0.524      0.429      0.509      0.356
#              minaret       1649         12      0.596      0.739      0.677      0.372
#           Motorbikes       1649        161      0.959      0.963      0.967      0.816
#             nautilus       1649          7       0.74      0.857      0.907       0.88
#              octopus       1649          6      0.663      0.657      0.627      0.577
#                okapi       1649         10      0.514        0.8      0.774      0.642
#               pagoda       1649         10      0.869        0.9      0.893      0.706
#                panda       1649          9      0.898       0.98      0.929      0.756
#               pigeon       1649          5      0.476        0.8      0.756        0.7
#                pizza       1649          9      0.842      0.889      0.849       0.81
#             platypus       1649          4      0.411       0.75      0.747      0.722
#              pyramid       1649         16       0.71       0.75      0.735      0.589
#             revolver       1649         15      0.862          1      0.995      0.885
#                rhino       1649          8       0.55       0.75      0.801      0.657
#              rooster       1649         17      0.916      0.882      0.955      0.841
#            saxophone       1649          9      0.682      0.667      0.652      0.529
#             schooner       1649         13      0.419      0.538      0.544       0.46
#             scissors       1649          6      0.763      0.833      0.807      0.768
#             scorpion       1649         20      0.671        0.7      0.744      0.592
#            sea_horse       1649         11      0.582      0.509      0.621      0.355
#               snoopy       1649         10      0.481        0.5        0.5      0.376
#          soccer_ball       1649         18      0.921      0.944      0.969      0.873
#              stapler       1649          5      0.678       0.85      0.938      0.884
#             starfish       1649         13      0.668      0.775      0.878      0.785
#          stegosaurus       1649         11       0.88          1      0.995      0.933
#            stop_sign       1649         15       0.64      0.733      0.833      0.716
#           strawberry       1649          4      0.333       0.75      0.695      0.655
#            sunflower       1649         11      0.499      0.818      0.634      0.473
#                 tick       1649         10       0.71        0.9      0.898        0.8
#            trilobite       1649         13      0.442      0.385      0.437      0.283
#             umbrella       1649         17      0.867      0.764      0.884      0.794
#                watch       1649         49       0.94      0.953      0.984      0.927
#          water_lilly       1649          7      0.473      0.135      0.286       0.21
#           wheelchair       1649         13      0.832      0.846      0.871      0.562
#             wild_cat       1649          7          1      0.243      0.848      0.739
#        windsor_chair       1649          8      0.807      0.875      0.923      0.888
#               wrench       1649          7      0.551      0.857      0.765      0.639
#             yin_yang       1649         15      0.845      0.728       0.86      0.816